In [2]:
import pyodbc
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po
model = pe.ConcreteModel("Cargacamiones")
import random
random.seed(0)

## Preprocesado de los datos

In [3]:
path_demanda=r"C:\Users\jmormal\PycharmProjects\pythonProject3\Demanda_validacion.xlsx"

In [4]:
df_demanda=pd.read_excel(path_demanda)

In [5]:
df_demanda=df_demanda.groupby("Collection in supplier").sum().iloc[: , :2]
df_demanda
df_demanda

,Magnum,A-cont
Collection in supplier,,
day 1,138,16
day 2,171,48
day 3,204,20
day 4,123,24
day 5,198,28
day 6,123,24
day 7,123,24
day 8,198,32
day 9,126,20


In [6]:
l=[]
for t in range(1,10):
    l1=[1,t,df_demanda.iloc[t-1 , 0 ]]
    l2=[2,t,df_demanda.iloc[t-1 , 1 ]]
    l.append(l1)
    l.append(l2)
df=pd.DataFrame(l,columns=["Producto","t","Demanda"])

In [7]:
path=r"C:\Users\jmormal\PycharmProjects\pythonProject3\Transporte_cubicaje_validacion.accdb"
conn = pyodbc.connect('Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+path+';')
cursor = conn.cursor()

Familias Carga de datos

In [8]:
cursor.execute('select IdProducto from Productos')
productos=[]
for row in cursor.fetchall():
    productos.append(row[0])

In [9]:
cursor.execute('select IdFamilia from Familias')
familias=[]
for row in cursor.fetchall():
    familias.append(row[0])

In [10]:
cursor.execute('select IdCamion from Camiones WHERE IdCamion<4')
camiones=[]
for i in range(1,7):
    camiones.append(i)

In [11]:
cursor.execute('select IdDia from Dias WHERE "IdDia"<=19')
dias=[]
for row in cursor.fetchall():
    dias.append(row[0])
        

In [12]:
cursor.execute('select IdDia from Dias')
dias1=[]
for row in cursor.fetchall():
    dias1.append(row[0])



In [13]:
columnas=[1,2]
productos

[2, 1]

## Familias Pyomo

In [14]:
model.i = pe.Set(initialize=productos)
model.f = pe.Set(initialize=familias)
model.c = pe.Set(initialize=camiones)
model.t = pe.Set(initialize=dias)
model.t1 = pe.Set(initialize=dias1)
model.col=pe.Set(initialize=columnas)


## Carga de datos productos

In [15]:
cursor.execute('select IdFamilia, LotePedido from Familias')
lotepedido={}
for row in cursor.fetchall():
    lotepedido.update({row[0]:row[1]})


In [16]:
cursor.execute('select IdFamilia, LotePedido from Familias')
CosteStock={}
for row in cursor.fetchall():
    CosteStock.update({row[0]:random.randint(7, 10)*0.001})


In [17]:
cursor.execute('select IdProducto , PiezasCont from Productos')
piezascont={}
for row in cursor.fetchall():
    piezascont.update({row[0]:row[1]})

In [18]:
cursor.execute('select IdProducto , LargoContedor from Productos')
piezas_largo_box={}
for row in cursor.fetchall():
    piezas_largo_box.update({row[0]:row[1]})

In [19]:
cursor.execute('select IdProducto , IdFamilia, beta from beta')
b={}
for row in cursor.fetchall():
    b.update({(row[0],row[1]):row[2]})


In [20]:
cursor.execute('select  IdProducto , IdDia, Demanda from Demanda')
demanda={}
for row in cursor.fetchall():
    # if row[0]>=13:
    #     demanda.update({(row[0],row[1]):row[2]})
    # else:
        demanda.update({(row[0],row[1]):row[2]})    

In [21]:
# cursor.execute('select IdProducto , StockInicial from Productos')
stockInicial={}
# for row in cursor.fetchall():
#     stockInicial.update({row[0]:row[1]})
cursor.execute('select  IdProducto , Demanda from Demanda where IdDia=1')

for row in cursor.fetchall():
    stockInicial.update({row[0]:row[1]*0})


In [22]:
model.i.__dict__["_init_values"]

In [23]:
cursor.execute('select  IdProducto , Valor from Productos')
valor={}
for row in cursor.fetchall():
    valor.update({row[0]:row[1]})

In [24]:
PC={}
for camion in camiones:
    PC.update({camion:1000})

In [25]:
L=78
nu=26/30
NT_lw=5
NT_up=10

In [26]:
model.LotePedido = pe.Param(model.f , initialize=lotepedido)
model.PiezasCont= pe.Param(model.i ,initialize=piezascont)
model.b= pe.Param(model.i, model.f ,initialize=b, default=0)
model.StockInicial= pe.Param(model.i, initialize=stockInicial)
model.demanda= pe.Param(model.i, model.t1,  initialize=demanda)
model.valor=pe.Param(model.i, initialize=stockInicial)
model.PC=pe.Param(model.c, initialize=PC)
model.L=pe.Param(initialize=L)
model.nu=pe.Param(initialize=nu)
model.NT_lw=pe.Param(initialize=NT_lw)
model.NT_up=pe.Param(initialize=NT_up)
model.LargoCont=pe.Param(model.i,initialize=piezas_largo_box)
model.CS=pe.Param(model.i, initialize=CosteStock)

# # Varibles

In [27]:
model.CantidadPedir = pe.Var(model.i, model.c, model.t,model.col, domain=pe.Integers)

In [28]:
model.CantidadPedir2 = pe.Var(model.i,model.f, model.c, model.t,model.col, domain=pe.Integers, bounds=(0,None) )
model.ContenedoresCamion=pe.Var(model.c,model.t,model.col,model.i, domain=pe.Integers)

In [29]:
model.Stock=pe.Var(model.i,model.t, bounds=(0,None))

In [30]:
model.RetrasoDemanda=pe.Var(model.i,model.t, bounds=(0,None))

In [31]:
model.K=pe.Var(model.f,model.c,model.t,model.col ,domain=pe.Integers)

In [32]:
model.Y=pe.Var(model.c,model.t ,domain=pe.Binary, initialize=0)

In [33]:
model.A=pe.Var(model.c,model.col,model.t, domain=pe.Integers, bounds=(0,4.5))
model.B=pe.Var(model.c,model.col, model.t, domain=pe.Integers, bounds=(0,12.5))
# model.A=pe.Var(model.c,model.col,model.t, domain=pe.Integers, bounds=(0,13))
# model.B=pe.Var(model.c,model.col, model.t, domain=pe.Integers, bounds=(0,13))


## Función Objetivo
Minimizar el número de camiones usado
$$\sum_{c,t}PC(c)Y(c,t)+ \sum CS(i)Stock(i,t)$$

In [34]:
expr = sum(model.PC[c]*model.Y[c,t] +sum(model.CS[i]*model.Stock[i,t] for i in model.i) for c in model.c for t in model.t)

In [35]:
 #Coste Transporte
model.objective = pe.Objective(sense=pe.minimize, expr=expr)

## Funciones auxiliares

In [36]:
def UsoCamion(model):
    return sum(model.L*model.Y[c,t] for c in model.c for t in model.t)-\
        sum(model.ContenedoresCamion[c,t] for c in model.c for t in model.t)

In [37]:
def CTransporte(model):
    return sum(model.PC[c]*model.Y[c,t] for c in model.c for t in model.t)

In [38]:
def NumeroCamiones(model):
    return sum(model.Y[c,t] for c in model.c for t in model.t)

## Restricciones

In [39]:
model.stocks=pe.ConstraintList()

$$Stock(i,t)=Stock(i,t-1)+\sum_{c,col}CantidadPedir(i,c,t,col)PiezasCont[i] -Demanda(i,t)$$

In [40]:
for i in model.i:
        lhs=model.Stock[i,1]
        rhs=model.StockInicial[i]-model.demanda[i,1]+sum(model.CantidadPedir[i,c,1,col]*model.PiezasCont[i] for c in model.c for col in model.col )
        model.stocks.add(lhs==rhs)

In [41]:
for t in list(model.t)[1:]:
    for i in model.i:
            lhs=model.Stock[i,t]
            rhs=model.Stock[i,t-1]-model.demanda[i,t]+sum(model.CantidadPedir[i,c,t,col]*model.PiezasCont[i] for c in model.c for col in model.col)
            model.stocks.add(lhs==rhs)

In [42]:
model.RcantidadPedir=pe.ConstraintList()

$$CantidadPedir2_{i,f,c,t,col}=K(f,c,t,col)*LotePedido(f)*b(i,f)$$

In [43]:
for i in model.i:
    for f in model.f:
        for c in model.c:
            for t in model.t:
                for col in model.col:
                    lhs=model.CantidadPedir2[i,f,c,t,col]
                    rhs=model.K[f,c,t,col]*model.LotePedido[f]*model.b[i,f]
                    model.RcantidadPedir.add(lhs==rhs)

In [44]:
model.RcantidadPedir2=pe.ConstraintList()

$$CantidadPedir_{i,c,t,col}= \sum_f CanitdadPedir2(i,f,c,t,col)$$

In [45]:
for i in model.i:
    for c in model.c:
        for t in model.t:
            for col in model.col:
                lhs=model.CantidadPedir[i,c,t,col]
                rhs=sum(model.CantidadPedir2[i,f,c,t,col] for f in model.f)
                model.RcantidadPedir2.add(lhs==rhs)

In [46]:
model.RDimensionesCamion=pe.ConstraintList()

$$ 1600A_{c,col,t} +1000B_{c,col,t}   \leq 13600* Y[c,t]$$

In [47]:
for c in model.c:
    for t in model.t:
        for col in model.col:
            lhs=13600*model.Y[c,t]
            rhs=(model.A[c,col,t]*1600+model.B[c,col,t]*1000)
            model.RDimensionesCamion.add(lhs>=rhs)

In [48]:
model.RCajasTotales=pe.ConstraintList()

$$ \forall c \in C \quad \forall t \in T \quad \sum_i CantidadPedir(i,c,t,col) \leq A_{c,col,t}*4+B_{c,col,t}*3 $$

In [49]:
for c in model.c:
    for t in model.t:
        for col in model.col:
            lhs=sum(model.CantidadPedir[i,c,t,col] for i in model.i)
            rhs=(model.A[c,col,t]*4+model.B[c,col,t]*3)
            model.RCajasTotales.add(lhs<=rhs)

In [50]:
model.RCajasTipoA=pe.ConstraintList()

$$ \sum_i CantidadPedir(i,c,t,col)_{tipo a}\leq A_{c,col,t}*4 $$

In [51]:
for c in model.c:
    for t in model.t:
        for col in model.col:
            lhs=sum(model.CantidadPedir[i,c,t,col]*float(int(model.LargoCont[i])==int(1600)) for i in model.i)
            rhs=model.A[c,col,t]*4
            model.RCajasTipoA.add(lhs<=rhs)

$$ \sum_i CantidadPedir(i,c,t,col)_{tipo b}\leq B_{c,col,t}*3 $$

In [52]:
model.RCajasTipoB=pe.ConstraintList()

In [53]:
for c in model.c:
    for t in model.t:
        for col in model.col:
            lhs=sum(model.CantidadPedir[i,c,t,col]*float(int(model.LargoCont[i])==int(1000)) for i in model.i)
            rhs=model.B[c,col,t]*3
            model.RCajasTipoB.add(lhs<=rhs)

In [54]:
model.RColumnasCamion=pe.ConstraintList()
for c in model.c:
    for t in model.t:
         model.RColumnasCamion.add(model.A[c,2,t]== model.A[c,1,t])
         model.RColumnasCamion.add(model.B[c,2,t]== model.B[c,1,t])


$$\forall i,t  \quad Stock[i,t]\geq \sum_{j \in [1,2,3,4]} Demanda[i,t+j]$$

In [55]:
model.RCobertura=pe.ConstraintList()
l=list(range(25))
for i in model.i:
    for t in range(1,7):
        lhs=model.Stock[i,t]
        #rhs=(model.demanda[i,t+1]+model.demanda[i,t+2]+model.demanda[i,t+3])
        rhs=0

        model.RCobertura.add(lhs>=rhs)
        l[i]=rhs
for i in model.i:
    for t in range(7,10):
        lhs=model.Stock[i,t]
        
        #rhs=l[i]
        rhs=0

        model.RCobertura.add(lhs>=rhs)


In [56]:
model.RCVolumenminimo=pe.ConstraintList()

In [57]:
for t in model.t:
    for c in model.c:
        lhs=(sum(model.CantidadPedir[i,c,t,col] for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))*750*1200*1600+
                                     +(sum(model.CantidadPedir[i,c,t,col] for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))*1000*1000*1200)/(3000*2400*13600)
        rhs=0.985*model.Y[c,t]
        model.RCVolumenminimo.add(lhs >= rhs)

In [58]:
model.bounds=pe.ConstraintList()
for i in model.i:
    for t in model.t:
        model.bounds.add(model.Stock[i,t]>=0)


In [59]:
solver = po.SolverFactory("gurobi")
solver.options["TimeLimit"]=60
# solver.options["MIPFocus"] = 0
solver.options["MIPGap"] = 0.01
results = solver.solve(model, tee=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-30
Read LP format model from file C:\Users\jmormal\AppData\Local\Temp\5\tmp1yf3hlnb.pyomo.lp
Reading time = 0.02 seconds
x1387: 1297 rows, 1153 columns, 3257 nonzeros
Set parameter TimeLimit to value 60
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 48 physical cores, 96 logical processors, using up to 32 threads
Optimize a model with 1297 rows, 1153 columns and 3257 nonzeros
Model fingerprint: 0xd1cfcac5
Variable types: 19 continuous, 1134 integer (54 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+04]
  Objective range  [6e-02, 1e+03]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 2e+02]
Presolve removed 955 rows and 757 columns
Presolve time: 0.02s
Presolved: 342 rows, 396 columns, 1114 nonzeros
Variable types: 0 continuous, 396 integer (54 binary)
Found heuristic solution: objective 54778.620000

Root relaxation

In [60]:
for t in range(1,6):
    print(t,model.Stock[i,t].value,-model.demanda[i,t+1]-model.demanda[i,t+2]-model.demanda[i,t+3] ,
          model.Stock[i,t].value-model.demanda[i,t+1]-model.demanda[i,t+2]-model.demanda[i,t+3])
    a=-model.demanda[i,t+1]-model.demanda[i,t+2]-model.demanda[i,t+3]-model.demanda[i,t+4]
for t in range(6,10):
    print(t,model.Stock[i,t].value,a ,model.Stock[i,t].value+a)
def mean(x):
    return sum(x)/len(x)

1 6.0 -498 -492.0
2 21.0 -525 -504.0
3 3.0 -444 -441.0
4 24.0 -444 -420.0
5 12.0 -444 -432.0
6 33.0 -570 -537.0
7 54.0 -570 -516.0
8 0.0 -570 -570.0
9 16.0 -570 -554.0


In [61]:
utilizacion=0
volumen_por_camion=[]
for t in model.t:
    for c in model.c:
        if model.Y[c,t].value==1:
            volumen_por_camion.append(((sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))*750*1200*1600+
                                     +(sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))*1000*1000*1200)/
                                     (3000*2400*13600)))
print(volumen_por_camion)
print(mean(volumen_por_camion))
print(len(volumen_por_camion))

[1.0, 1.0, 1.0, 0.9852941176470589, 1.0, 0.9852941176470589, 0.9852941176470589, 0.9852941176470589, 0.9852941176470589, 0.9852941176470589, 0.9852941176470589, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9877450980392157, 0.9877450980392157]
0.9939309056956115
21


In [62]:
for t in model.t:
    print(sum(model.Y[c,t].value for c in model.c))

2.0
3.0
3.0
2.0
3.0
2.0
2.0
2.0
2.0


In [63]:
l=[]
for t in model.t:
    for i in model.i:
        1
        # print(t,i,model.Stock[i,t].value, sum(model.Y[c,t].value  for c in model.c if model.CantidadPedir[i,c,t,col].value>0))

            # if model.Y[c,t].value==1:
            #     print(t,model.A[c,col,t].value,model.B[c,col,t].value,((sum(model.CantidadPedir[i,c,t,col].value/model.PiezasCont[i] for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))*750*1200*1600+
            #                          +(sum(model.CantidadPedir[i,c,t,col].value/model.PiezasCont[i] for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))*1000*1000*1200)/
            #                          (3000*2400*13600)))

In [64]:
l=[]
for t in model.t:
    for c in model.c:
        for col in model.col:

            if model.Y[c,t].value==1:
                l.append([c,col,t,model.A[c,col,t].value,model.B[c,col,t].value,((sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))*750*1200*1600+
                                     +(sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))*1000*1000*1200)/
                                     (3000*2400*13600),[model.CantidadPedir[i,c,t,col].value for i in model.i])])
                # print([model.CantidadPedir[i,c,t,col].value for i in model.i])
                # print(t,model.A[c,col,t].value,model.B[c,col,t].value,((sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))*750*1200*1600+
                #                      +(sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))*1000*1000*1200)/
                #                      (3000*2400*13600)),
                #     sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))
                #       ,sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))

In [65]:
df1=pd.DataFrame(l,columns=["c", "col", "t", "A", "B", "Volumen",])
df1

,c,col,t,A,B,Volumen
0,1,1,1,1.0,12.0,"(1.0, [4.0, 36.0])"
1,1,2,1,1.0,12.0,"(1.0, [4.0, 36.0])"
2,2,1,1,1.0,12.0,"(1.0, [4.0, 36.0])"
3,2,2,1,1.0,12.0,"(1.0, [4.0, 36.0])"
4,2,1,2,1.0,12.0,"(1.0, [4.0, 36.0])"
5,2,2,2,1.0,12.0,"(1.0, [4.0, 36.0])"
6,4,1,2,4.0,7.0,"(0.9852941176470589, [16.0, 21.0])"
7,4,2,2,4.0,7.0,"(0.9852941176470589, [16.0, 21.0])"
8,5,1,2,1.0,12.0,"(1.0, [4.0, 36.0])"
9,5,2,2,1.0,12.0,"(1.0, [4.0, 36.0])"


In [66]:
sum([model.ContenedoresCamion[c,t,col,i].value*float(int(model.LargoCont[i])==int(1000)) for i in model.i])<=model.B[c,col,t].value*3

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

In [ ]:
for t in model.t:
    for c in model.c:
        for i in model.i:
            print(model.CantidadPedir[i,c,t,col].value)#*float(int(model.LargoCont[i])==int(1000)))

In [ ]:
l1=[]
a=list(range(25))
for t in range(1,7):
    for i in model.i:
            l1.append([t,i,model.Stock[i,t].value,-model.demanda[i,t+1]-model.demanda[i,t+2]-model.demanda[i,t+3] ,
          model.Stock[i,t].value-model.demanda[i,t+1]-model.demanda[i,t+2]-model.demanda[i,t+3]])
            a[i]=-model.demanda[i,t+1]-model.demanda[i,t+2]-model.demanda[i,t+3]
for t in range(7,9):
    for i in model.i:
            l1.append([t,i,model.Stock[i,t].value,a[i] ,
          model.Stock[i,t].value+a[i]])
a

In [ ]:
df2=pd.DataFrame(l1,columns=["t","id","Stock","Demanda Proximos Dias","Diferencia Demanda"])
pd.set_option('display.max_rows', None)
df2

In [ ]:
# for t in model.t:
#     print(t,sum(model.CantidadPedir[i,c,t,col].value for c in model.c for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))
print(sum(model.CantidadPedir[i,c,t,col].value for t in model.t for c in model.c for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600)))
print(sum(model.CantidadPedir[i,c,t,col].value for t in model.t for c in model.c for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))

In [ ]:
df_demanda

In [ ]:
for i,v in model.LargoCont:
    print(i)

In [ ]:
sum(model.demanda[i,t]/ for )

In [ ]:
import scip